In [ ]:
import os
os.chdir(r"C:\Users\AGN's SP\OneDrive\课件\数据可视化\期末\work")

# 爬虫

In [ ]:
from selenium import webdriver
browser = webdriver.Chrome()
dict_={}
for i in range(10):
    dict_[i+2009]={}
    browser.get('http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-year-{}-0-1-1'.format(str(i+2009)))
    browser.find_elements
    list_ = browser.find_elements_by_xpath("/html/body/div[3]/div[3]/div[2]/ul/li")
    for index, l in enumerate(list_):
        name = browser.find_element_by_xpath("/html/body/div[3]/div[3]/div[2]/ul/li[{}]/div[@class='name']".format(index+1)).text
        #print(name)
        comment = browser.find_element_by_xpath("/html/body/div[3]/div[3]/div[2]/ul/li[{}]/div[@class='star']".format(index+1)).text
        #print(comment)
        author = browser.find_element_by_xpath("/html/body/div[3]/div[3]/div[2]/ul/li[{}]/div[5]".format(index+1)).text
        #print(author)
        cbs = browser.find_element_by_xpath("/html/body/div[3]/div[3]/div[2]/ul/li[{}]/div[6]".format(index+1)).text
        #print(cbs)
        price = browser.find_element_by_xpath("/html/body/div[3]/div[3]/div[2]/ul/li[{}]/div[@class='price']/p/span".format(index+1)).text
        #print(price)
        dict_[i+2009][name]={'author':author,'comment':comment,'price':price, 'cbs':cbs,\
                         'url':browser.find_element_by_partial_link_text(name).get_attribute('href')}

In [ ]:
dict_

In [ ]:
import collections
books = []
for key in dict_.keys():
    books.extend(dict_[key].keys())
counter = collections.Counter(books)
most_20 = counter.most_common(20)
dict_20 = {}
for common in most_20:
    bookname = common[0]
    #print(bookname)
    dict_20[bookname] = {}
    dict_20[bookname]['id'] = None
    year = 2009
    while dict_20[bookname]['id']==None:
        if bookname in dict_[year].keys():
            dict_20[bookname]['id']=dict_[year][bookname]['url'].replace('http://product.dangdang.com/','').replace('.html', '')
            dict_20[bookname]['comment']=int(dict_[year][bookname]['comment'].split('条')[0])
        year += 1
dict_20

In [ ]:
import requests
import json
import time
from bs4 import BeautifulSoup as bs
id_ = 20026456
page = 1
for book in dict_20.keys():
    print('-'*100)
    print(book,' start!')
    dict_20[book]['comments']=[]
    id_ = dict_20[book]['id']
    headers={
            'Host': 'product.dangdang.com',
            'Referer': "http://product.dangdang.com/{}.html".format(id_),
            'User-Agent': "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
        }
    response = requests.get("http://product.dangdang.com/index.php?r=comment%2Flist&productId={}&\
                                    categoryPath=01.18.10.00.00.00&mainProductId={}&mediumId=0&pageIndex={}&sortType=1&filterType=1\
                                    &isSystem=1&tagId=0&tagFilterCount=0&template=publish&long_or_short=short".format(id_,id_,1), headers=headers)
    js = json.loads(response.text)
    dict_20[book]['comments_num'] = int(js['data']['list']['summary']['total_comment_num'])
    num_page = 1000
    if int(dict_20[book]['comments_num']/10)<1000:
        num_page = int(dict_20[book]['comments_num']/10)
    for page in range(num_page):
        print("第{}页".format(page))
        page += 1
        #comments = []
        #times = []
        try:
            response = requests.get("http://product.dangdang.com/index.php?r=comment%2Flist&productId={}&\
                                    categoryPath=01.18.10.00.00.00&mainProductId={}&mediumId=0&pageIndex={}&sortType=1&filterType=1\
                                    &isSystem=1&tagId=0&tagFilterCount=0&template=publish&long_or_short=short".format(id_,id_,page), headers=headers)
            #print(response)
            js = json.loads(response.text)
            soup = bs(js['data']['list']['html'], 'lxml')

            lst = soup.find_all('div', class_='items_right')
            for l in lst:
                comment = []
                for n in l.find_all('span'):
                    if ''.join(n.text.split())!='':
                        comment.append(n.text)
                #comments.append(comment[0])
                for m in comment:
                    if m[:2]=='20':
                        dict_20[book]['comments'].append((comment[0], m))
                        #times.append(m)
                        break
        except:
            print("第{}页失败".format(page))
        time.sleep(1)


In [ ]:
f = open(r"data\dict_20.txt",'w',encoding='utf-8')
f.write(str(dict_20))
f.close()

# 数据分析

In [ ]:
import os
os.chdir(r"C:\Users\AGN's SP\OneDrive\课件\数据可视化\期末\work")

In [2]:
#读取
f = open(r"data\dict_20.txt",'r',encoding='utf-8')
a = f.read()
dict_20 = eval(a)
f.close()

In [3]:
from collections import defaultdict
import re
def time_count(dict_):
    comments = dict_['comments']
    comments_month = defaultdict(list)
    for n in comments:
        if len(re.findall('20..-..-',n[1]))==0:
            continue
        month = n[1][:7].replace('-','')
        comments_month[month].append(n[0])
    comments_month = sorted(comments_month.items(),key=lambda x:x[0])
    return comments_month

#dict_ = time_count(dict_20['百年孤独（加西亚马尔克斯代表作）'])

In [ ]:
from pyecharts.charts import Bar
import pyecharts.options as opts
#import matplotlib.pyplot as plt
bar = Bar()#新建柱状图
bar.add_xaxis((list(dict_20.keys())).add_yaxis("评论数", [dict_20[name]['comment'] for name in dict_20.keys()])    # 增加y轴
bar.set_global_opts(=opts.AxisTickOpts())
bar.render('总评论数.html')

# 情感分析 存为csv

In [ ]:
import pandas as pd

def dict2csv(dict_):
    data = []
    index = 0
    for bookname in dict_.keys():
        for n in dict_[bookname]['comments']:
            if len(re.findall('20..-..-',n[1]))!=0 and n[1].strip()!='':
                data.append([n[1],bookname,n[0]])
    data = pd.DataFrame(data, columns=['日期','书名','评论'])
    return data
data = dict2csv(dict_20)
#data.to_csv(r"data/book_comments_20.csv", index=0,encoding='utf-8')

In [ ]:
import snownlp 
import tqdm
def senti(data):
    senti_score = []
    keywords = []
    summary = []
    for n in tqdm.tqdm_notebook(data['评论']):
        s = snownlp.SnowNLP(n)
        senti_score.append(s.sentiments)
        keywords.append(' '.join(s.keywords()))
        try:
            summary.append(s.summary(1)[0])
        except:
            summary.append('-')
    data['情感分数']=senti_scoree
    data['关键词']=keywords
    data['总结']=summary
senti(data)
data.to_csv(r"data/book_comments_20.csv", index=0,encoding='utf-8')

# 评论数量折线图

In [ ]:
import pandas as pd
import matplotlib
import scipy
import matplotlib.pyplot as plt
from matplotlib.font_manager import *
from datetime import datetime
import matplotlib.dates as mdates

import pyecharts.options as opts
from example.commons import Collector, Faker
from pyecharts.charts import Line, Page

def lineDraw(info):
    A = Collector()
    @A.funcs
    def line() -> Line:
        date = [n[0] for n in info]
        bookname=info[0][1]
        comments_num = [n[2] for n in info]
        line = (
            Line(init_opts=opts.InitOpts(bg_color="white"))
            .add_xaxis(date)    # 增加x轴
            .add_yaxis("评论数", comments_num, is_smooth=True,
                       areastyle_opts=opts.AreaStyleOpts(opacity=0.5))     # 增加y轴
            #.add_yaxis("情感分数", senti_score, is_smooth=True,
            #           areastyle_opts=opts.AreaStyleOpts(opacity=0.5))
            .set_global_opts(
                title_opts=opts.TitleOpts(title=bookname, subtitle="评论数"),
                xaxis_opts=opts.AxisOpts(
                    axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
                    is_scale=False,
                    boundary_gap=False
                ),  # 图像贴近y轴
                yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True))   # 增加y轴分割线
            )   # 全局配置项
            .set_series_opts(
                markpoint_opts=opts.MarkPointOpts(
                    data=[opts.MarkPointItem(type_="max"), opts.MarkPointItem(type_="min")],
                    symbol_size=[34, 30],
                    label_opts=opts.LabelOpts(position="inside", color="#fff", font_size=8)
                ),
                label_opts=opts.LabelOpts(is_show=False),
                markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")])
            )   # 系列配置项
        )
        return line
    Page().add(*[fn() for fn, _ in A.charts]).render(u'./line.html')

In [4]:
def show_num_all():
    com_num = []
    for bookname in dict_20.keys():
        lst = []
        dict_ = time_count(dict_20[bookname])
        comments_month_num = defaultdict(int)
        for n in dict_:
            lst.append([n[0][0:4]+'-'+n[0][4:6], bookname, len(n[1])])
        #lineDraw(lst)    #画line取消注释
        com_num.extend(lst)
    return com_num
com_num = show_num_all()

In [5]:
date =sorted(list(set([x[0][0:4]+'-'+x[0][4:6] for x in com_num])))
bookname = list(set([n[1] for n in com_num]))
data = sorted(com_num, key=lambda x:x[0])
data= [[x[0][0:4]+'-'+x[0][4:6],x[1],x[2]] for x in data]

# 评论数量3D图

In [ ]:
from example.commons import Collector, Faker
from pyecharts import options as opts
from pyecharts.charts import Bar3D, Page
import random
C = Collector()
@C.funcs
def bar3d_base() -> Bar3D:
    # data = [(i, j, random.randint(0, 12)) for i in range(6) for j in range(24)]
    # print([[d[1], d[0], d[2]] for d in data])
    c = (
        Bar3D(init_opts=opts.InitOpts(width='1200px', height='1000px'))
        .add(
            "",
            data,
            xaxis3d_opts=opts.Axis3DOpts(date, type_="category"),
            yaxis3d_opts=opts.Axis3DOpts(bookname, type_="category"),
            zaxis3d_opts=opts.Axis3DOpts(type_="value"),
        )
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(max_=500),
            title_opts=opts.TitleOpts(title="评论数量3D"),
        )
    )
    return c

p = Page()
p.add(*[fn() for fn, _ in C.charts]).render(u'./评论数量3Dbar.html')

# 情感分析饼状图

In [6]:
import pandas as pd
df = pd.read_csv(r"data/book_comments_20.csv")

In [7]:
df[:5]

,日期,书名,评论,情感分数,关键词,总结
0,2012-02-19 15:21:07,百年孤独（加西亚马尔克斯代表作）,对我而言，单从读《百年孤独》的直观感觉来说，是非常的微妙且有意思的。与以往容易入书...,1.0,孤独 层 良诺 奥雷 总,到最后一个具有家族明显特征的奥雷良诺
1,2013-03-03 13:42:01,百年孤独（加西亚马尔克斯代表作）,第一遍看这本书的时候真的是迷迷糊糊的，一点也看不懂，只是照着情节读了下来，字里行间里就是有一...,1.0,里 看 种 透 印象,一点也看不懂
2,2013-12-06 22:04:19,百年孤独（加西亚马尔克斯代表作）,《百年孤独》（西班牙语：Cien a?os de soledad），是哥伦比亚作家加夫列尔·...,1.0,百年 年 孤独 拉丁美洲 马尔克斯,”《百年孤独》发表于1967年
3,2012-12-19 21:00:36,百年孤独（加西亚马尔克斯代表作）,《百年孤独》，是哥伦比亚作家加西亚·马尔克斯的代表作，也是拉丁美洲魔幻现实主义文学的代表作...,1.0,家族 历史 人 此书 都,布恩迪亚家族的每个人都渴望得到幸福
4,2011-06-20 19:07:30,百年孤独（加西亚马尔克斯代表作）,整本书简直是一首磅礴华丽的交响乐！\r作者一开始就营造了长期与世隔绝的小村落所散发的安定却极...,1.0,恩迪亚 布 孤独 家族 却,但最大的孤独感恐怕出于布恩迪亚家族自身


In [8]:
import pandas as pd
from numpy import max, min, mean, quantile, median, percentile

from example.commons import Collector, Faker
from pyecharts import options as opts
from pyecharts.charts import Page, Pie

def pie(data, bookname):
    C = Collector()
    @C.funcs
    def pie_rosetype() -> Pie:
        c = (
            Pie()
            .add('', data,
            label_opts=opts.LabelOpts(is_show=True),)
            .set_global_opts(title_opts=opts.TitleOpts(title=bookname,subtitle='情感分布'))
        )
        return c
    Page().add(*[fn() for fn, _ in C.charts]).render(u'./pie-情感分析.html')

def senti_score_show(dftotal):
    for name in bookname:
        #print(name)
        df = dftotal[dftotal['书名']==name]
        #  winsorize 去除极端值
        q5 = df.quantile(0.05)['情感分数']
        q95 = df.quantile(0.95)['情感分数']
        # print(df_weibo.quantile(0.95)['情感得分'])
        senti_list = []
        for i in df['情感分数']:
            if (i <= q95) and (i >= q5):
                senti_list.append(i)

        senti_dict = {}
        # 分位数统计，统计结果有用于人工将情感得分划分出区间
        senti_max = max(senti_list)
        senti_min = min(senti_list)
        senti_mean = mean(senti_list)
        senti_25 = (senti_mean - senti_min) / 2
        senti_75 = (senti_max - senti_mean) / 2
        #print(senti_max, senti_min, senti_mean, senti_25, senti_75)
        senti_dict['极端负面情感']=0
        senti_dict['负面情感']=0
        senti_dict['中性情感']=0
        senti_dict['正向情感']=0
        senti_dict['极端正向情感']=0
        for i in senti_list:
            if i < 0.1:
                senti_dict['极端负面情感'] = senti_dict.get("极端负面情感", 0) + 1
            elif 0.1 <= i < 0.4:
                senti_dict['负面情感'] = senti_dict.get("负面情感", 0) + 1
            elif 0.4 <= i < 0.6:
                senti_dict['中性情感'] = senti_dict.get("中性情感", 0) + 1
            elif 0.6 <= i < 0.9:
                senti_dict['正向情感'] = senti_dict.get("正向情感", 0) + 1
            elif 0.9 <= i:
                senti_dict['极端正向情感'] = senti_dict.get("极端正向情感", 0) + 1

        data = []
        for k, v in senti_dict.items():
            data1 = []
            data1.append(k)
            data1.append(v)
            data.append(data1)
        #print(data)
        pie(data,name)

senti_score_show(df)

# 情感分析3D图

In [ ]:
import tqdm
df['日期'] = df['日期'].apply(lambda x:x[:7])
date_month = sorted(list(set([n for n in df['日期']])))
senscore = []
for name in tqdm.tqdm_notebook(bookname):
    for month in date_month:
        ddf = df[df['书名']==name][df['日期']==month]['情感分数']
        try:
            senscore.append([month, name, sum(ddf)/len(ddf)])
        except:
            continue

In [ ]:
senscore[:5]

In [ ]:
from example.commons import Collector, Faker
from pyecharts import options as opts
from pyecharts.charts import Bar3D, Page
import random
C = Collector()
@C.funcs
def bar3d_base() -> Bar3D:
    # data = [(i, j, random.randint(0, 12)) for i in range(6) for j in range(24)]
    # print([[d[1], d[0], d[2]] for d in data])
    c = (
        Bar3D(init_opts=opts.InitOpts(width='1200px', height='1000px'))
        .add(
            "",
            senscore,
            xaxis3d_opts=opts.Axis3DOpts(date_month, type_="category"),
            yaxis3d_opts=opts.Axis3DOpts(bookname, type_="category"),
            zaxis3d_opts=opts.Axis3DOpts(type_="value"),
        )
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(max_=1),
            title_opts=opts.TitleOpts(title="情感分数3D"),
        )
    )
    return c

p = Page()
p.add(*[fn() for fn, _ in C.charts]).render(u'./情感分数3Dbar.html')

# 关键词

In [ ]:
import collections
def getstopword(path):
    with open(path, encoding='utf-8') as f:
        stopword = [line.strip('\n') for line in f.readlines()]
        stopword.append(' ')
        stopword.append('\xa0')
        stopword.append('\n')
        stopword.append('\t')
    return list(set(stopword))
stopword_path = r"stopwords.txt"
stopwords = getstopword(stopword_path)

book_keywords_20 = []
for name in bookname:
    key = df['关键词'][df['书名']==name]
    keywords = []
    for n in key:
        #print(n)
        try:
            keywords.extend([n for n in n.split() if n not in stopwords])
        except:
            True
    count = collections.Counter(keywords)
    keywords_20 = []
    keywords_20.append(name)
    keywords_20.extend([n[0] for n in count.most_common(20)])
    book_keywords_20.append(keywords_20)
    #print(name, count.most_common(20))

In [ ]:
columns = ['书名']
columns.extend(['第{}个关键词'.format(m+1)for m in range(20)])
df_keywords_20 = pd.DataFrame(book_keywords_20, columns=columns)

In [ ]:
df_keywords_20.to_csv('keywords_20.csv', index=0)

# 关键词知识图谱构建

In [ ]:
from py2neo import Graph,Node, Relationship,NodeMatcher
g = Graph(host="127.0.0.1",  # neo4j 搭载服务器的ip地址，ifconfig可获取到
            http_port=7474,  # neo4j 服务器监听的端口号
            user="neo4j",  # 数据库user name，如果没有更改过，应该是neo4j
            password="140801")


In [ ]:
bookname = list(set(df_keywords_20['书名']))
keywords = []
for n in range(20):
    keywords.extend(df_keywords_20['第{}个关键词'.format(n+1)])
keywords = list(set(keywords))

In [ ]:
for keyword in keywords:
    if keyword in bookname:
        continue
    node = Node(keyword, value=keyword, name=keyword,type_='keyword')
    g.create(node)
for i, name in enumerate(bookname):
    node = Node(name, value=name, name=name, type_='book')
    g.create(node) 
    for n in range(10):
        word = df_keywords_20[df_keywords_20['书名']==name]['第{}个关键词'.format(n+1)].iloc[0]
        if name!=word:
            query = "match(p),(q) where p.name='%s'and q.value='%s' create (q)-[rel:%s{name:'%s'}]->(p)" % (
                           name, word, 'comments', 'keyword{}'.format(n+1))
            g.run(query)
g.run("MATCH (n) where n.type_='book' RETURN n")